In [51]:
import csv
import pandas as pd 
import numpy as np
from sklearn import linear_model
import statsmodels.api as sm
import pyodbc


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
################################
# For Rates per 60 
################################

In [71]:
################################
# Even Strength sample data
################################
import pyodbc 
pd.set_option('display.max_columns', None)
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-TS4R5T7\SQLEXPRESS;'
                      'Database=NHL;'
                      'Trusted_Connection=yes;')

# cursor = conn.cursor()
# cursor.execute('SELECT * FROM [dbo].[Master Even Strength]')

ES_For_Sample = pd.read_sql_query('SELECT * FROM [NHL].[dbo].[Master Even Strength] WHERE [NHL].[dbo].[Master Even Strength].[Date] < '+"'2021-08-08'"+'ORDER BY Year, Month, Day',conn)

ES_For_Column_Means = ES_For_Sample.mean()
ES_For_Sample = ES_For_Sample.fillna(ES_For_Column_Means)
count_nan_in_ES_For_Sample = ES_For_Sample.isnull().sum()

ES_For_Sample.head()

,GameID,Location,Opponent,Day,Month,Year,Date,Team,GP,TOI_GP,W,L,OTL,ROW,Points,Point,CF_60,CA_60,CF,FF_60,FA_60,FF,SF_60,SA_60,SF,GF_60,GA_60,GF,xGF_60,xGA_60,xGF,SCF_60,SCA_60,SCF,SCSF_60,SCSA_60,SCSF,SCGF_60,SCGA_60,SCGF,SCSH,SCSV,HDCF_60,HDCA_60,HDCF,HDSF_60,HDSA_60,HDSF,HDGF_60,HDGA_60,HDGF,HDSH,HDSV,MDCF_60,MDCA_60,MDCF,MDSF_60,MDSA_60,MDSF,MDGF_60,MDGA_60,MDGF,MDSH,MDSV,LDCF_60,LDCA_60,LDCF,LDSF_60,LDSA_60,LDSF,LDGF_60,LDGA_60,LDGF,LDSH,LDSV,SH,SV,PDO
0,04-Oct-2017 Toronto Maple Leafs,Away,Winnipeg Jets,4,10,2017,2017-10-04,Toronto Maple Leafs,1,39.133301,True,False,False,True,2,1.0,56.730000,58.259998,49.330002,36.799999,36.799999,50.000000,26.059999,27.600000,48.570000,7.67,3.07,71.430000,2.11,2.39,46.860001,30.660000,24.530001,55.560001,18.400000,16.870001,52.169998,6.13,3.07,66.669998,33.330002,81.820000,10.73,12.27,46.669998,9.20,12.27,42.860001,3.07,3.07,50.0,33.330002,75.000000,19.930000,12.270000,61.900002,9.20,4.60,66.669998,3.07,0.00,100.0,33.330002,100.000000,19.930000,24.530001,44.830002,6.13,9.200000,40.00,1.53,0.00,100.0,25.0,100.0,29.41,88.889999,1.183
1,04-Oct-2017 Winnipeg Jets,Home,Toronto Maple Leafs,4,10,2017,2017-10-04,Winnipeg Jets,1,39.133301,False,True,False,False,0,0.0,58.259998,56.730000,50.669998,36.799999,36.799999,50.000000,27.600000,26.059999,51.430000,3.07,7.67,28.570000,2.39,2.11,53.139999,24.530001,30.660000,44.439999,16.870001,18.400000,47.830002,3.07,6.13,33.330002,18.180000,66.669998,12.27,10.73,53.330002,12.27,9.20,57.139999,3.07,3.07,50.0,25.000000,66.669998,12.270000,19.930000,38.099998,4.60,9.20,33.330002,0.00,3.07,0.0,0.000000,66.669998,24.530001,19.930000,55.169998,9.20,6.130000,60.00,0.00,1.53,0.0,0.0,75.0,11.11,70.589996,0.817
2,04-Oct-2017 St Louis Blues,Away,Pittsburgh Penguins,4,10,2017,2017-10-04,St Louis Blues,1,49.566700,True,False,False,True,2,1.0,60.520000,73.839996,45.049999,48.419998,55.680000,46.509998,35.099998,36.310001,49.150002,4.84,3.63,57.139999,2.92,3.52,45.340000,32.680000,33.889999,49.090000,23.000000,14.530000,61.290001,4.84,0.00,100.000000,21.049999,100.000000,13.32,12.10,52.380001,10.89,4.84,69.230003,1.21,0.00,100.0,11.110000,100.000000,19.370001,21.790001,47.060001,12.10,9.68,55.560001,3.63,0.00,100.0,30.000000,100.000000,25.420000,35.099998,42.000000,12.10,18.160000,40.00,0.00,3.63,0.0,0.0,80.0,13.79,90.000000,1.038
3,04-Oct-2017 Pittsburgh Penguins,Home,St Louis Blues,4,10,2017,2017-10-04,Pittsburgh Penguins,1,49.566700,False,False,True,False,1,0.5,73.839996,60.520000,54.950001,55.680000,48.419998,53.490002,36.310001,35.099998,50.849998,3.63,4.84,42.860001,3.52,2.92,54.660000,33.889999,32.680000,50.910000,14.530000,23.000000,38.709999,0.00,4.84,0.000000,0.000000,78.949997,12.10,13.32,47.619999,4.84,10.89,30.770000,0.00,1.21,0.0,0.000000,88.889999,21.790001,19.370001,52.939999,9.68,12.10,44.439999,0.00,3.63,0.0,0.000000,70.000000,35.099998,25.420000,58.000000,18.16,12.100000,60.00,3.63,0.00,100.0,20.0,100.0,10.00,86.209999,0.962
4,04-Oct-2017 Calgary Flames,Away,Edmonton Oilers,4,10,2017,2017-10-04,Calgary Flames,1,54.000000,False,True,False,False,0,0.0,60.000000,81.110001,42.520000,42.220001,65.559998,39.180000,27.780001,46.669998,37.310001,0.00,3.33,0.000000,2.16,5.68,27.510000,22.219999,33.330002,40.000000,12.220000,22.219999,35.480000,0.00,3.33,0.000000,0.000000,85.000000,10.00,20.00,33.330002,7.78,15.56,33.330002,0.00,2.22,0.0,0.000000,85.709999,12.220000,13.330000,47.830002,4.44,6.67,40.000000,0.00,1.11,0.0,0.000000,83.330002,24.440001,41.110001,37.290001,11.11,24.440001,31.25,0.00,0.00,50.0,0.0,100.0,0.00,92.860001,0.929


In [74]:
ES_For_X = ES_For_Sample[['FF', 'SF_60','SCSF_60','SCSF','SCGF','SCSH','HDGF_60', 
        'MDGF_60', 'LDCF_60','LDSF_60','LDGF_60','LDSH','SH']]
ES_For_Y = ES_For_Sample['GF_60']
 

# with sklearn
ES_For_Regression = linear_model.LinearRegression()
ES_For_Regression.fit(ES_For_X, ES_For_Y)

#print('Intercept: \n', regr.intercept_)
#print('Coefficients: \n', regr.coef_)

# with statsmodels
ES_For_X = sm.add_constant(ES_For_X) # adding a constant
 
ES_For_Model = sm.OLS(ES_For_Y, ES_For_X).fit()
predictions = ES_For_Model.predict(ES_For_X) 
 
ES_For_Print_Model = ES_For_Model.summary()
# print(print_model)
# for x in range (0, len(regr.coef_)):
#     print(model.pvalues[x])
ES_For_Coefficients = pd.DataFrame(ES_For_Model.params,columns=['COEFFICIENTS'])
print(ES_For_Print_Model)

                            OLS Regression Results                            
Dep. Variable:                  GF_60   R-squared:                       0.984
Model:                            OLS   Adj. R-squared:                  0.984
Method:                 Least Squares   F-statistic:                 4.366e+04
Date:                Mon, 10 Oct 2022   Prob (F-statistic):               0.00
Time:                        09:12:49   Log-Likelihood:                 597.53
No. Observations:                8984   AIC:                            -1167.
Df Residuals:                    8970   BIC:                            -1068.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.6737      0.020    -33.274      0.0

In [75]:
import pyodbc 
pd.set_option('display.max_columns', None)
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-TS4R5T7\SQLEXPRESS;'
                      'Database=NHL;'
                      'Trusted_Connection=yes;')

# cursor = conn.cursor()
# cursor.execute('SELECT * FROM [dbo].[Master Even Strength]')

ES_For_Test = pd.read_sql_query('SELECT * FROM [NHL].[dbo].[Master Even Strength] WHERE [NHL].[dbo].[Master Even Strength].[Date] > '+"'2021-08-08'"+'ORDER BY Year, Month, Day',conn)

ES_For_Column_Means = ES_For_Test.mean()
ES_For_Test = ES_For_Test.fillna(ES_For_Column_Means)
count_nan_in_ES_For_Test = ES_For_Test.isnull().sum()

ES_For_Test['xG'] = ES_For_Coefficients.iat[0,0] + (EStest['FF']*ES_For_Coefficients.iat[1,0] + EStest['SF_60']*ES_For_Coefficients.iat[2,0] + 
                                      EStest['SCSF_60']*ES_For_Coefficients.iat[3,0] +  EStest['SCSF']*ES_For_Coefficients.iat[4,0] +
                                      EStest['SCGF']*ES_For_Coefficients.iat[5,0] + EStest['SCSH']*ES_For_Coefficients.iat[6,0] +
                                      EStest['HDGF_60']*ES_For_Coefficients.iat[7,0] + EStest['MDGF_60']*ES_For_Coefficients.iat[8,0] +
                                      EStest['LDCF_60']*ES_For_Coefficients.iat[9,0] + EStest['LDSF_60']*ES_For_Coefficients.iat[10,0] +
                                      EStest['LDGF_60']*ES_For_Coefficients.iat[11,0] + EStest['LDSH']*ES_For_Coefficients.iat[12,0] +
                                      EStest['SH']*ES_For_Coefficients.iat[13,0] 
                                     )
print("Expected Goals for with Even Strength:",ES_For_Test['xG'].mean())
print("True Goals for with Even Strength:",ES_For_Test['GF_60'].mean())
print("Average difference:", ES_For_Test['GF_60'].mean()-ES_For_Test['xG'].mean())


Expected Goals for with Even Strength: 2.8405494825011055
True Goals for with Even Strength: 2.8582964976113745
Average difference: 0.017747015110268993


In [81]:
################################
# Penalty Kill sample data 
################################
import pyodbc 
pd.set_option('display.max_columns', None)
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-TS4R5T7\SQLEXPRESS;'
                      'Database=NHL;'
                      'Trusted_Connection=yes;')

# cursor = conn.cursor()
# cursor.execute('SELECT * FROM [dbo].[Master Even Strength]')

PK_For_Sample = pd.read_sql_query('SELECT * FROM [NHL].[dbo].[Master Penalty Kill] WHERE [NHL].[dbo].[Master Penalty Kill].[Date] < '+"'2021-08-08'"+'ORDER BY Year, Month, Day',conn)

PK_For_Column_Means = PK_For_Sample.mean()
PK_For_Sample = PK_For_Sample.fillna(PK_For_Column_Means)
count_nan_in_PK_For_Sample = PK_For_Sample.isnull().sum()

PK_For_Sample.head()


,GameID,Location,Opponent,Day,Month,Year,Date,Team,GP,TOI_GP,W,L,OTL,ROW,Points,Point,CF_60,CA_60,CF,FF_60,FA_60,FF,SF_60,SA_60,SF,GF_60,GA_60,GF,xGF_60,xGA_60,xGF,SCF_60,SCA_60,SCF,SCSF_60,SCSA_60,SCSF,SCGF_60,SCGA_60,SCGF,SCSH,SCSV,HDCF_60,HDCA_60,HDCF,HDSF_60,HDSA_60,HDSF,HDGF_60,HDGA_60,HDGF,HDSH,HDSV,MDCF_60,MDCA_60,MDCF,MDSF_60,MDSA_60,MDSF,MDGF_60,MDGA_60,MDGF,MDSH,MDSV,LDCF_60,LDCA_60,LDCF,LDSF_60,LDSA_60,LDSF,LDGF_60,LDGA_60,LDGF,LDSH,LDSV,SH,SV,PDO
0,04-Oct-2017 Toronto Maple Leafs,Away,Winnipeg Jets,4,10,2017,2017-10-04,Toronto Maple Leafs,1,16.0000,True,False,False,True,2,1.0,15.000000,116.250000,11.43,11.25,82.500000,12.00,11.25,67.500000,14.29,0.0,0.00,12.058397,0.02,6.04,0.27,0.0,63.750000,0.0,0.0,30.000000,0.0,0.0,0.00,13.245983,-,100.000000,0.0,30.000000,0.0,0.0,22.500000,0.0,0.0,0.00,-,-,100.000000,0.0,33.750000,0.00,0.0,7.500000,0.000000,0.0,0.00,-,-,100.0,3.75,48.750000,7.14,0.0,37.50,0.000000,0.0,0.0,-,-,100.00000,0.000000,100.000000,1.000000
1,04-Oct-2017 Winnipeg Jets,Home,Toronto Maple Leafs,4,10,2017,2017-10-04,Winnipeg Jets,1,4.8667,False,True,False,False,0,0.0,24.660000,147.949997,14.29,12.33,147.949997,7.69,12.33,135.619995,8.33,0.0,24.66,0.000000,0.02,23.02,0.10,0.0,110.959999,0.0,0.0,110.959999,0.0,0.0,24.66,0.000000,-,77.779999,0.0,73.970001,0.0,0.0,73.970001,0.0,0.0,24.66,0,-,66.669998,0.0,36.990002,0.00,0.0,36.990002,0.000000,0.0,0.00,-,-,100.0,12.33,36.990002,25.00,0.0,24.66,0.000000,0.0,0.0,-,-,100.00000,0.000000,81.820000,0.818000
2,04-Oct-2017 Pittsburgh Penguins,Home,St Louis Blues,4,10,2017,2017-10-04,Pittsburgh Penguins,1,5.3667,False,False,True,False,1,0.5,22.360001,55.900002,28.57,11.18,55.900002,16.67,0.00,44.720001,0.00,0.0,11.18,0.000000,0.63,6.94,8.35,0.0,44.720001,0.0,0.0,44.720001,0.0,0.0,11.18,0.000000,-,75.000000,0.0,22.360001,0.0,0.0,22.360001,0.0,0.0,0.00,-,-,100.000000,0.0,22.360001,0.00,0.0,22.360001,0.000000,0.0,11.18,0,-,50.0,11.18,11.180000,50.00,0.0,0.00,8.251617,0.0,0.0,-,-,91.85137,9.273519,75.000000,0.949369
3,04-Oct-2017 St Louis Blues,Away,Pittsburgh Penguins,4,10,2017,2017-10-04,St Louis Blues,1,6.3167,True,False,False,True,2,1.0,9.500000,85.489998,10.00,9.50,56.990002,14.29,9.50,28.500000,25.00,0.0,9.50,0.000000,0.62,9.74,6.00,9.5,66.489998,12.5,9.5,28.500000,25.0,0.0,9.50,0.000000,0,66.669998,0.0,9.500000,0.0,0.0,9.500000,0.0,0.0,9.50,0,-,0.000000,9.5,56.990002,14.29,9.5,19.000000,33.330002,0.0,0.00,-,0,100.0,0.00,19.000000,0.00,0.0,0.00,8.251617,0.0,0.0,-,-,91.85137,0.000000,66.669998,0.667000
4,04-Oct-2017 Calgary Flames,Away,Edmonton Oilers,4,10,2017,2017-10-04,Calgary Flames,1,4.0000,False,True,False,False,0,0.0,30.000000,75.000000,28.57,15.00,45.000000,25.00,15.00,45.000000,25.00,0.0,0.00,12.058397,2.57,3.60,41.66,15.0,15.000000,50.0,15.0,15.000000,50.0,0.0,0.00,13.245983,0,100.000000,15.0,0.000000,100.0,15.0,0.000000,100.0,0.0,0.00,-,0,75.342440,0.0,15.000000,0.00,0.0,15.000000,0.000000,0.0,0.00,-,-,100.0,15.00,60.000000,20.00,0.0,30.00,0.000000,0.0,0.0,-,-,100.00000,0.000000,100.000000,1.000000


In [82]:
PK_For_X = PK_For_Sample[['FF', 'SF_60','SCSF_60','SCSF','SCGF','HDGF_60', 
        'MDGF_60','LDSF_60','LDGF_60','SH']]

PK_For_Y = PK_For_Sample['GF_60']
 

# with sklearn
PK_For_Regression = linear_model.LinearRegression()
PK_For_Regression.fit(PK_For_X, PK_For_Y)


#print('Intercept: \n', regr.intercept_)
#print('Coefficients: \n', regr.coef_)

# with statsmodels
PK_For_X = sm.add_constant(PK_For_X) # adding a constant
 
PK_For_Model = sm.OLS(PK_For_Y, PK_For_X).fit()
predictions = PK_For_Model.predict(PK_For_X) 
 
PK_For_Print_Model = PK_For_Model.summary()
# print(print_model)
# for x in range (0, len(regr.coef_)):
#     print(model.pvalues[x])
PK_For_Coefficients = pd.DataFrame(PK_For_Model.params,columns=['COEFFICIENTS'])
print(PK_For_Print_Model)

                            OLS Regression Results                            
Dep. Variable:                  GF_60   R-squared:                       0.959
Model:                            OLS   Adj. R-squared:                  0.958
Method:                 Least Squares   F-statistic:                 2.036e+04
Date:                Mon, 10 Oct 2022   Prob (F-statistic):               0.00
Time:                        09:22:03   Log-Likelihood:                -10696.
No. Observations:                8822   AIC:                         2.141e+04
Df Residuals:                    8811   BIC:                         2.149e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2249      0.014    -16.303      0.0

In [84]:
from constrained_linear_regression import ConstrainedLinearRegression
from sklearn.linear_model import LinearRegression
import pyodbc 
pd.set_option('display.max_columns', None)
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-TS4R5T7\SQLEXPRESS;'
                      'Database=NHL;'
                      'Trusted_Connection=yes;')

# cursor = conn.cursor()
# cursor.execute('SELECT * FROM [dbo].[Master Even Strength]')

PK_For_Test = pd.read_sql_query('SELECT * FROM [NHL].[dbo].[Master Penalty Kill] WHERE [NHL].[dbo].[Master Penalty Kill].[Date] > '+"'2021-08-08'"+'ORDER BY Year, Month, Day',conn)

PK_For_Column_Means = PK_For_Test.mean()
PK_For_Test = PKtest.fillna(PK_For_Column_Means)
count_nan_in_PK_For_Test = PK_For_Test.isnull().sum()

PK_For_Test['xG'] = PK_For_Coefficients.iat[0,0] + (PKtest['FF']*PK_For_Coefficients.iat[1,0] + PKtest['SF_60']*PK_For_Coefficients.iat[2,0] + 
                                      PKtest['SCSF_60']*PK_For_Coefficients.iat[3,0] +  PKtest['SCSF']*PK_For_Coefficients.iat[4,0] +
                                      PKtest['SCGF']*PK_For_Coefficients.iat[5,0] +
                                      PKtest['HDGF_60']*PK_For_Coefficients.iat[6,0] + PKtest['MDGF_60']*PK_For_Coefficients.iat[7,0] +
                                      PKtest['LDSF_60']*PK_For_Coefficients.iat[8,0] +
                                      PKtest['LDGF_60']*PK_For_Coefficients.iat[9,0] + 
                                      PKtest['SH']*PK_For_Coefficients.iat[10,0] 
                                     )


print("Expected Goals for with Penalty Kill:",PK_For_Test['xG'].mean())
print("True Goals for with Penalty Kill:",PK_For_Test['GF_60'].mean())
print("Average difference:", PK_For_Test['GF_60'].mean()-PK_For_Test['xG'].mean())



Expected Goals for with Penalty Kill: 0.9259732396947867
True Goals for with Penalty Kill: 0.9215627411937053
Average difference: -0.004410498501081439


In [86]:
################################
# Power Play sample data 
################################
import pyodbc 
pd.set_option('display.max_columns', None)
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-TS4R5T7\SQLEXPRESS;'
                      'Database=NHL;'
                      'Trusted_Connection=yes;')

# cursor = conn.cursor()
# cursor.execute('SELECT * FROM [dbo].[Master Even Strength]')

PP_For_Sample = pd.read_sql_query('SELECT * FROM [NHL].[dbo].[Master Power Play] WHERE [NHL].[dbo].[Master Power Play].[Date] < '+"'2021-08-08'"+'ORDER BY Year, Month, Day',conn)

PP_For_Column_Means = PP_For_Sample.mean()
PP_For_Sample = PP_For_Sample.fillna(PP_For_Column_Means)
count_nan_in_PP_For_Sample = PP_For_Sample.isnull().sum()

PP_For_Sample.head()

,GameID,Location,Opponent,Day,Month,Year,Date,Team,GP,TOI_GP,W,L,OTL,ROW,Points,Point,CF_60,CA_60,CF,FF_60,FA_60,FF,SF_60,SA_60,SF,GF_60,GA_60,GF,xGF_60,xGA_60,xGF,SCF_60,SCA_60,SCF,SCSF_60,SCSA_60,SCSF,SCGF_60,SCGA_60,SCGF,SCSH,SCSV,HDCF_60,HDCA_60,HDCF,HDSF_60,HDSA_60,HDSF,HDGF_60,HDGA_60,HDGF,HDSH,HDSV,MDCF_60,MDCA_60,MDCF,MDSF_60,MDSA_60,MDSF,MDGF_60,MDGA_60,MDGF,MDSH,MDSV,LDCF_60,LDCA_60,LDCF,LDSF_60,LDSA_60,LDSF,LDGF_60,LDGA_60,LDGF,LDSH,LDSV,SH,SV,PDO
0,04-Oct-2017 Toronto Maple Leafs,Away,Winnipeg Jets,4,10,2017,2017-10-04,Toronto Maple Leafs,1,4.8667,True,False,False,True,2,1.0,147.949997,24.660000,85.709999,147.949997,12.33,92.309998,135.619995,12.33,91.669998,24.66,0.0,100.000000,23.02,0.02,99.900002,110.959999,0.0,100.0,110.959999,0.0,100.0,24.66,0.0,100.000000,22.219999,-,73.970001,0.0,100.0,73.970001,0.0,100.0,24.66,0.0,100,33.330002,-,36.990002,0.0,100.000000,36.990002,0.0,100.000000,0.00,0.0,-,0.000000,-,36.990002,12.33,75.000000,24.66,0.0,100.000000,0.0,0.0,-,0.00000,-,18.180000,100.000000,1.182000
1,04-Oct-2017 Winnipeg Jets,Home,Toronto Maple Leafs,4,10,2017,2017-10-04,Winnipeg Jets,1,16.0000,False,True,False,False,0,0.0,116.250000,15.000000,88.570000,82.500000,11.25,88.000000,67.500000,11.25,85.709999,0.00,0.0,87.941603,6.04,0.02,99.730003,63.750000,0.0,100.0,30.000000,0.0,100.0,0.00,0.0,86.754017,0.000000,-,30.000000,0.0,100.0,22.500000,0.0,100.0,0.00,0.0,-,0.000000,-,33.750000,0.0,100.000000,7.500000,0.0,100.000000,0.00,0.0,-,0.000000,-,48.750000,3.75,92.860001,37.50,0.0,100.000000,0.0,0.0,-,0.00000,-,0.000000,100.000000,1.000000
2,04-Oct-2017 St Louis Blues,Away,Pittsburgh Penguins,4,10,2017,2017-10-04,St Louis Blues,1,5.3667,True,False,False,True,2,1.0,55.900002,22.360001,71.430000,55.900002,11.18,83.330002,44.720001,0.00,100.000000,11.18,0.0,100.000000,6.94,0.63,91.650002,44.720001,0.0,100.0,44.720001,0.0,100.0,11.18,0.0,100.000000,25.000000,-,22.360001,0.0,100.0,22.360001,0.0,100.0,0.00,0.0,-,0.000000,-,22.360001,0.0,100.000000,22.360001,0.0,100.000000,11.18,0.0,100,50.000000,-,11.180000,11.18,50.000000,0.00,0.0,91.748383,0.0,0.0,-,8.14863,-,25.000000,90.726481,1.050632
3,04-Oct-2017 Pittsburgh Penguins,Home,St Louis Blues,4,10,2017,2017-10-04,Pittsburgh Penguins,1,6.3167,False,False,True,False,1,0.5,85.489998,9.500000,90.000000,56.990002,9.50,85.709999,28.500000,9.50,75.000000,9.50,0.0,100.000000,9.74,0.62,94.000000,66.489998,9.5,87.5,28.500000,9.5,75.0,9.50,0.0,100.000000,33.330002,100,9.500000,0.0,100.0,9.500000,0.0,100.0,9.50,0.0,100,100.000000,-,56.990002,9.5,85.709999,19.000000,9.5,66.669998,0.00,0.0,-,0.000000,100,19.000000,0.00,100.000000,0.00,0.0,91.748383,0.0,0.0,-,8.14863,-,33.330002,100.000000,1.333000
4,04-Oct-2017 Calgary Flames,Away,Edmonton Oilers,4,10,2017,2017-10-04,Calgary Flames,1,2.0000,False,True,False,False,0,0.0,60.000000,60.000000,50.000000,30.000000,30.00,50.000000,30.000000,0.00,100.000000,0.00,0.0,87.941603,8.71,1.92,81.919998,30.000000,0.0,100.0,30.000000,0.0,100.0,0.00,0.0,86.754017,0.000000,-,30.000000,0.0,100.0,30.000000,0.0,100.0,0.00,0.0,-,0.000000,-,0.000000,0.0,88.299621,0.000000,0.0,86.072079,0.00,0.0,-,14.980242,-,30.000000,30.00,50.000000,0.00,0.0,91.748383,0.0,0.0,-,8.14863,-,0.000000,90.726481,1.050632


In [87]:
PP_For_X = PP_For_Sample[['SF_60','SCSF_60','HDGF_60', 
        'MDGF_60','LDGF_60','SH']]

PP_For_Y = PP_For_Sample['GF_60']
 

# with sklearn
PP_For_Regression = linear_model.LinearRegression()
PP_For_Regression.fit(PP_For_X, PP_For_Y)

#print('Intercept: \n', regr.intercept_)
#print('Coefficients: \n', regr.coef_)

# with statsmodels
PP_For_X = sm.add_constant(PP_For_X) # adding a constant
 
PP_For_Model = sm.OLS(PP_For_Y, PP_For_X).fit()
predictions = PP_For_Model.predict(PP_For_X) 
 
PP_For_Print_Model = PP_For_Model.summary()
# print(print_model)
# for x in range (0, len(regr.coef_)):
#     print(model.pvalues[x])
PP_For_Coefficients = pd.DataFrame(PP_For_Model.params,columns=['COEFFICIENTS'])
PP_For_Print_Model


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  GF_60   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 1.546e+06
Date:                Mon, 10 Oct 2022   Prob (F-statistic):               0.00
Time:                        09:32:08   Log-Likelihood:                -12978.
No. Observations:                8822   AIC:                         2.597e+04
Df Residuals:                    8815   BIC:                         2.602e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1391      0.024     -5.894      0.000      -0.185      -0.093
SF_60          0.0047      0.001      8.372      0.000       0.004       0.006
SCSF_60       -0.0036      0.001     -5.197      0.000      -0.005      -0.002
HDGF_60        0.9966      0.001   1334.168      0.000       0.995       0.998
MDGF_60        0.9953      0.001    924.191      0.000       0.993       0.997
LDGF_60        0.9941      0.001   1242.197      0.000       0.993       0.996
SH             0.0053      0.001      8.466      0.000       0.004       0.007
==============================================================================
Omnibus:                    22474.731   Durbin-Watson:                   1.971
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        384594363.510
Skew:                          28.603   Prob(JB):                         0.00
Kurtosis:                    1024.276   Cond. No.                         195.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [96]:
import pyodbc 
pd.set_option('display.max_columns', None)
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-TS4R5T7\SQLEXPRESS;'
                      'Database=NHL;'
                      'Trusted_Connection=yes;')

# cursor = conn.cursor()
# cursor.execute('SELECT * FROM [dbo].[Master Even Strength]')

PP_For_Test = pd.read_sql_query('SELECT * FROM [NHL].[dbo].[Master Power Play] WHERE [NHL].[dbo].[Master Power Play].[Date] > '+"'2021-08-08'"+'ORDER BY Year, Month, Day',conn)

# column_means = PPtest.mean()
# PPtest = PPtest.fillna(column_means)
# count_nan_in_df = PPtest.isnull().sum()

PP_For_Test['xG'] = PP_For_Coefficients.iat[0,0] + ( PP_For_Test['SF_60']*PP_For_Coefficients.iat[1,0] + 
                                      PP_For_Test['SCSF_60']*PP_For_Coefficients.iat[2,0] + 
                                      PP_For_Test['HDGF_60']*PP_For_Coefficients.iat[3,0] + 
                                      PP_For_Test['MDGF_60']*PP_For_Coefficients.iat[4,0] +
                                      PP_For_Test['LDGF_60']*PP_For_Coefficients.iat[5,0] + 
                                      PP_For_Test['SH']*PP_For_Coefficients.iat[6,0] 
                                     )

print("Expected Goals for with Penalty Power Play:",PP_For_Test['xG'].mean())
print("True Goals for with Power Play:",PP_For_Test['GF_60'].mean())
print("Average difference:", PP_For_Test['GF_60'].mean()-PP_For_Test['xG'].mean())


Expected Goals for with Penalty Power Play: 11.860522419599837
True Goals for with Power Play: 11.15962388155857
Average difference: -0.7008985380412671


In [ ]:
################################
################################
# Against Rates per 60 
################################
################################

In [109]:
################################
# Even Strength sample data 
################################
import pyodbc 
pd.set_option('display.max_columns', None)
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-TS4R5T7\SQLEXPRESS;'
                      'Database=NHL;'
                      'Trusted_Connection=yes;')

# cursor = conn.cursor()
# cursor.execute('SELECT * FROM [dbo].[Master Even Strength]')

ES_Against_Sample = pd.read_sql_query('SELECT * FROM [NHL].[dbo].[Master Even Strength] WHERE [NHL].[dbo].[Master Even Strength].[Date] < '+"'2021-08-08'"+'ORDER BY Year, Month, Day',conn)

ES_Against_Column_Means = ES_Against_Sample.mean()
ES_Against_Sample = ES_Against_Sample.fillna(ES_Against_Column_Means)
count_nan_in_ES_Against_Sample = ES_Against_Sample.isnull().sum()

ES_Against_Sample.head()

,GameID,Location,Opponent,Day,Month,Year,Date,Team,GP,TOI_GP,W,L,OTL,ROW,Points,Point,CF_60,CA_60,CF,FF_60,FA_60,FF,SF_60,SA_60,SF,GF_60,GA_60,GF,xGF_60,xGA_60,xGF,SCF_60,SCA_60,SCF,SCSF_60,SCSA_60,SCSF,SCGF_60,SCGA_60,SCGF,SCSH,SCSV,HDCF_60,HDCA_60,HDCF,HDSF_60,HDSA_60,HDSF,HDGF_60,HDGA_60,HDGF,HDSH,HDSV,MDCF_60,MDCA_60,MDCF,MDSF_60,MDSA_60,MDSF,MDGF_60,MDGA_60,MDGF,MDSH,MDSV,LDCF_60,LDCA_60,LDCF,LDSF_60,LDSA_60,LDSF,LDGF_60,LDGA_60,LDGF,LDSH,LDSV,SH,SV,PDO
0,04-Oct-2017 Toronto Maple Leafs,Away,Winnipeg Jets,4,10,2017,2017-10-04,Toronto Maple Leafs,1,39.133301,True,False,False,True,2,1.0,56.730000,58.259998,49.330002,36.799999,36.799999,50.000000,26.059999,27.600000,48.570000,7.67,3.07,71.430000,2.11,2.39,46.860001,30.660000,24.530001,55.560001,18.400000,16.870001,52.169998,6.13,3.07,66.669998,33.330002,81.820000,10.73,12.27,46.669998,9.20,12.27,42.860001,3.07,3.07,50.0,33.330002,75.000000,19.930000,12.270000,61.900002,9.20,4.60,66.669998,3.07,0.00,100.0,33.330002,100.000000,19.930000,24.530001,44.830002,6.13,9.200000,40.00,1.53,0.00,100.0,25.0,100.0,29.41,88.889999,1.183
1,04-Oct-2017 Winnipeg Jets,Home,Toronto Maple Leafs,4,10,2017,2017-10-04,Winnipeg Jets,1,39.133301,False,True,False,False,0,0.0,58.259998,56.730000,50.669998,36.799999,36.799999,50.000000,27.600000,26.059999,51.430000,3.07,7.67,28.570000,2.39,2.11,53.139999,24.530001,30.660000,44.439999,16.870001,18.400000,47.830002,3.07,6.13,33.330002,18.180000,66.669998,12.27,10.73,53.330002,12.27,9.20,57.139999,3.07,3.07,50.0,25.000000,66.669998,12.270000,19.930000,38.099998,4.60,9.20,33.330002,0.00,3.07,0.0,0.000000,66.669998,24.530001,19.930000,55.169998,9.20,6.130000,60.00,0.00,1.53,0.0,0.0,75.0,11.11,70.589996,0.817
2,04-Oct-2017 St Louis Blues,Away,Pittsburgh Penguins,4,10,2017,2017-10-04,St Louis Blues,1,49.566700,True,False,False,True,2,1.0,60.520000,73.839996,45.049999,48.419998,55.680000,46.509998,35.099998,36.310001,49.150002,4.84,3.63,57.139999,2.92,3.52,45.340000,32.680000,33.889999,49.090000,23.000000,14.530000,61.290001,4.84,0.00,100.000000,21.049999,100.000000,13.32,12.10,52.380001,10.89,4.84,69.230003,1.21,0.00,100.0,11.110000,100.000000,19.370001,21.790001,47.060001,12.10,9.68,55.560001,3.63,0.00,100.0,30.000000,100.000000,25.420000,35.099998,42.000000,12.10,18.160000,40.00,0.00,3.63,0.0,0.0,80.0,13.79,90.000000,1.038
3,04-Oct-2017 Pittsburgh Penguins,Home,St Louis Blues,4,10,2017,2017-10-04,Pittsburgh Penguins,1,49.566700,False,False,True,False,1,0.5,73.839996,60.520000,54.950001,55.680000,48.419998,53.490002,36.310001,35.099998,50.849998,3.63,4.84,42.860001,3.52,2.92,54.660000,33.889999,32.680000,50.910000,14.530000,23.000000,38.709999,0.00,4.84,0.000000,0.000000,78.949997,12.10,13.32,47.619999,4.84,10.89,30.770000,0.00,1.21,0.0,0.000000,88.889999,21.790001,19.370001,52.939999,9.68,12.10,44.439999,0.00,3.63,0.0,0.000000,70.000000,35.099998,25.420000,58.000000,18.16,12.100000,60.00,3.63,0.00,100.0,20.0,100.0,10.00,86.209999,0.962
4,04-Oct-2017 Calgary Flames,Away,Edmonton Oilers,4,10,2017,2017-10-04,Calgary Flames,1,54.000000,False,True,False,False,0,0.0,60.000000,81.110001,42.520000,42.220001,65.559998,39.180000,27.780001,46.669998,37.310001,0.00,3.33,0.000000,2.16,5.68,27.510000,22.219999,33.330002,40.000000,12.220000,22.219999,35.480000,0.00,3.33,0.000000,0.000000,85.000000,10.00,20.00,33.330002,7.78,15.56,33.330002,0.00,2.22,0.0,0.000000,85.709999,12.220000,13.330000,47.830002,4.44,6.67,40.000000,0.00,1.11,0.0,0.000000,83.330002,24.440001,41.110001,37.290001,11.11,24.440001,31.25,0.00,0.00,50.0,0.0,100.0,0.00,92.860001,0.929


In [117]:
###########
# Even Strength 
###########

ES_Against_X = ES_Against_Sample[['SA_60','SCSV','HDSA_60','HDGA_60','MDSA_60','MDGA_60','LDGA_60','SV']]
                        
ES_Against_Y = ES_Against_Sample['GA_60']
 

# with sklearn
ES_Against_Regression = linear_model.LinearRegression()
ES_Against_Regression.fit(ES_Against_X, ES_Against_Y)

#print('Intercept: \n', regr.intercept_)
#print('Coefficients: \n', regr.coef_)

# with statsmodels
ES_Against_X = sm.add_constant(ES_Against_X) # adding a constant
 
ES_Against_Model = sm.OLS(ES_Against_Y, ES_Against_X).fit()
ES_Against_Predictions = ES_Against_Model.predict(ES_Against_X) 
 
ES_Against_Print_Model = ES_Against_Model.summary()
# print(print_model)
# for x in range (0, len(regr.coef_)):
#     print(model.pvalues[x])
ES_Against_Coefficients = pd.DataFrame(ES_Against_Model.params,columns=['COEFFICIENTS'])
print(ES_Against_Print_Model)

                            OLS Regression Results                            
Dep. Variable:                  GA_60   R-squared:                       0.983
Model:                            OLS   Adj. R-squared:                  0.983
Method:                 Least Squares   F-statistic:                 6.456e+04
Date:                Mon, 10 Oct 2022   Prob (F-statistic):               0.00
Time:                        10:21:45   Log-Likelihood:                 177.47
No. Observations:                8984   AIC:                            -336.9
Df Residuals:                    8975   BIC:                            -273.0
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          8.8445      0.125     70.558      0.0

In [118]:
###
# Testing analysis against most recent season game data
###
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-TS4R5T7\SQLEXPRESS;'
                      'Database=NHL;'
                      'Trusted_Connection=yes;')

# cursor = conn.cursor()
# cursor.execute('SELECT * FROM [dbo].[Master Even Strength]')

ES_Against_Test = pd.read_sql_query('SELECT * FROM [NHL].[dbo].[Master Even Strength] WHERE [NHL].[dbo].[Master Even Strength].[Date] > '+"'2021-08-08'"+'ORDER BY Year, Month, Day',conn)

ES_Against_Column_Means = ES_Against_Test.mean()
ES_Against_Test = ES_Against_Test.fillna(ES_Against_Column_Means)
count_nan_in_df = ES_Against_Test.isnull().sum()

ES_Against_Test['xG'] = ES_Against_Coefficients.iat[0,0] + (ES_Against_Test['SA_60']*ES_Against_Coefficients.iat[1,0] + 
                                                            ES_Against_Test['SCSV']*ES_Against_Coefficients.iat[2,0] + 
                                                            ES_Against_Test['HDSA_60']*ES_Against_Coefficients.iat[3,0] +  
                                                            ES_Against_Test['HDGA_60']*ES_Against_Coefficients.iat[4,0] +
                                                            ES_Against_Test['MDSA_60']*ES_Against_Coefficients.iat[5,0] + 
                                                            ES_Against_Test['MDGA_60']*ES_Against_Coefficients.iat[6,0] +
                                                            ES_Against_Test['LDGA_60']*ES_Against_Coefficients.iat[7,0] +
                                                            ES_Against_Test['SV']*ES_Against_Coefficients.iat[8,0])

print("Expected Goals against with Even Strength:",ES_Against_Test['xG'].mean())
print("True Goals for with Even Strength:",ES_Against_Test['GF_60'].mean())
print("Average difference:", ES_Against_Test['GF_60'].mean()-ES_Against_Test['xG'].mean())



Expected Goals against with Even Strength: 2.842756825676379
True Goals for with Even Strength: 2.8582964976113745
Average difference: 0.015539671934995614


In [112]:
################################
# Penalty Kill sample data 
################################
import pyodbc 
pd.set_option('display.max_columns', None)
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-TS4R5T7\SQLEXPRESS;'
                      'Database=NHL;'
                      'Trusted_Connection=yes;')

# cursor = conn.cursor()
# cursor.execute('SELECT * FROM [dbo].[Master Even Strength]')

PK_Against_Sample = pd.read_sql_query('SELECT * FROM [NHL].[dbo].[Master Penalty Kill] WHERE [NHL].[dbo].[Master Penalty Kill].[Date] < '+"'2021-08-08'"+'ORDER BY Year, Month, Day',conn)

PK_Against_Column_Means = PK_Against_Sample.mean()
PK_Against_Sample = PK_Against_Sample.fillna(PK_Against_Column_Means)
count_nan_in_PK_Against_Sample = PK_Against_Sample.isnull().sum()

PK_Against_Sample.head()

,GameID,Location,Opponent,Day,Month,Year,Date,Team,GP,TOI_GP,W,L,OTL,ROW,Points,Point,CF_60,CA_60,CF,FF_60,FA_60,FF,SF_60,SA_60,SF,GF_60,GA_60,GF,xGF_60,xGA_60,xGF,SCF_60,SCA_60,SCF,SCSF_60,SCSA_60,SCSF,SCGF_60,SCGA_60,SCGF,SCSH,SCSV,HDCF_60,HDCA_60,HDCF,HDSF_60,HDSA_60,HDSF,HDGF_60,HDGA_60,HDGF,HDSH,HDSV,MDCF_60,MDCA_60,MDCF,MDSF_60,MDSA_60,MDSF,MDGF_60,MDGA_60,MDGF,MDSH,MDSV,LDCF_60,LDCA_60,LDCF,LDSF_60,LDSA_60,LDSF,LDGF_60,LDGA_60,LDGF,LDSH,LDSV,SH,SV,PDO
0,04-Oct-2017 Toronto Maple Leafs,Away,Winnipeg Jets,4,10,2017,2017-10-04,Toronto Maple Leafs,1,16.0000,True,False,False,True,2,1.0,15.000000,116.250000,11.43,11.25,82.500000,12.00,11.25,67.500000,14.29,0.0,0.00,12.058397,0.02,6.04,0.27,0.0,63.750000,0.0,0.0,30.000000,0.0,0.0,0.00,13.245983,-,100.000000,0.0,30.000000,0.0,0.0,22.500000,0.0,0.0,0.00,-,-,100.000000,0.0,33.750000,0.00,0.0,7.500000,0.000000,0.0,0.00,-,-,100.0,3.75,48.750000,7.14,0.0,37.50,0.000000,0.0,0.0,-,-,100.00000,0.000000,100.000000,1.000000
1,04-Oct-2017 Winnipeg Jets,Home,Toronto Maple Leafs,4,10,2017,2017-10-04,Winnipeg Jets,1,4.8667,False,True,False,False,0,0.0,24.660000,147.949997,14.29,12.33,147.949997,7.69,12.33,135.619995,8.33,0.0,24.66,0.000000,0.02,23.02,0.10,0.0,110.959999,0.0,0.0,110.959999,0.0,0.0,24.66,0.000000,-,77.779999,0.0,73.970001,0.0,0.0,73.970001,0.0,0.0,24.66,0,-,66.669998,0.0,36.990002,0.00,0.0,36.990002,0.000000,0.0,0.00,-,-,100.0,12.33,36.990002,25.00,0.0,24.66,0.000000,0.0,0.0,-,-,100.00000,0.000000,81.820000,0.818000
2,04-Oct-2017 Pittsburgh Penguins,Home,St Louis Blues,4,10,2017,2017-10-04,Pittsburgh Penguins,1,5.3667,False,False,True,False,1,0.5,22.360001,55.900002,28.57,11.18,55.900002,16.67,0.00,44.720001,0.00,0.0,11.18,0.000000,0.63,6.94,8.35,0.0,44.720001,0.0,0.0,44.720001,0.0,0.0,11.18,0.000000,-,75.000000,0.0,22.360001,0.0,0.0,22.360001,0.0,0.0,0.00,-,-,100.000000,0.0,22.360001,0.00,0.0,22.360001,0.000000,0.0,11.18,0,-,50.0,11.18,11.180000,50.00,0.0,0.00,8.251617,0.0,0.0,-,-,91.85137,9.273519,75.000000,0.949369
3,04-Oct-2017 St Louis Blues,Away,Pittsburgh Penguins,4,10,2017,2017-10-04,St Louis Blues,1,6.3167,True,False,False,True,2,1.0,9.500000,85.489998,10.00,9.50,56.990002,14.29,9.50,28.500000,25.00,0.0,9.50,0.000000,0.62,9.74,6.00,9.5,66.489998,12.5,9.5,28.500000,25.0,0.0,9.50,0.000000,0,66.669998,0.0,9.500000,0.0,0.0,9.500000,0.0,0.0,9.50,0,-,0.000000,9.5,56.990002,14.29,9.5,19.000000,33.330002,0.0,0.00,-,0,100.0,0.00,19.000000,0.00,0.0,0.00,8.251617,0.0,0.0,-,-,91.85137,0.000000,66.669998,0.667000
4,04-Oct-2017 Calgary Flames,Away,Edmonton Oilers,4,10,2017,2017-10-04,Calgary Flames,1,4.0000,False,True,False,False,0,0.0,30.000000,75.000000,28.57,15.00,45.000000,25.00,15.00,45.000000,25.00,0.0,0.00,12.058397,2.57,3.60,41.66,15.0,15.000000,50.0,15.0,15.000000,50.0,0.0,0.00,13.245983,0,100.000000,15.0,0.000000,100.0,15.0,0.000000,100.0,0.0,0.00,-,0,75.342440,0.0,15.000000,0.00,0.0,15.000000,0.000000,0.0,0.00,-,-,100.0,15.00,60.000000,20.00,0.0,30.00,0.000000,0.0,0.0,-,-,100.00000,0.000000,100.000000,1.000000


In [119]:
###########
# Penalty Kill 
###########

PK_Against_X = PK_Against_Sample[['SA_60','SCSV','HDSA_60','HDGA_60','MDSA_60','MDGA_60','LDGA_60','SV']]
                        
PK_Against_Y = PK_Against_Sample['GA_60']
 

# with sklearn
PK_Against_Regression = linear_model.LinearRegression()
PK_Against_Regression.fit(PK_Against_X, PK_Against_Y)

#print('Intercept: \n', regr.intercept_)
#print('Coefficients: \n', regr.coef_)

# with statsmodels
PK_Against_X = sm.add_constant(PK_Against_X) # adding a constant
 
PK_Against_Model = sm.OLS(PK_Against_Y, PK_Against_X).fit()
PK_Against_Predictions = PK_Against_Model.predict(PK_Against_X) 
 
PK_Against_Print_Model = PK_Against_Model.summary()
# print(print_model)
# for x in range (0, len(regr.coef_)):
#     print(model.pvalues[x])
PK_Against_Coefficients = pd.DataFrame(PK_Against_Model.params,columns=['COEFFICIENTS'])
print(PK_Against_Print_Model)

                            OLS Regression Results                            
Dep. Variable:                  GA_60   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 1.167e+06
Date:                Mon, 10 Oct 2022   Prob (F-statistic):               0.00
Time:                        10:23:23   Log-Likelihood:                -12948.
No. Observations:                8822   AIC:                         2.591e+04
Df Residuals:                    8813   BIC:                         2.598e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3728      0.053      7.050      0.0

In [120]:
###
# Testing analysis against most recent season game data
###
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-TS4R5T7\SQLEXPRESS;'
                      'Database=NHL;'
                      'Trusted_Connection=yes;')

# cursor = conn.cursor()
# cursor.execute('SELECT * FROM [dbo].[Master Even Strength]')

PK_Against_Test = pd.read_sql_query('SELECT * FROM [NHL].[dbo].[Master Penalty Kill] WHERE [NHL].[dbo].[Master Penalty Kill].[Date] > '+"'2021-08-08'"+'ORDER BY Year, Month, Day',conn)

PK_Against_Column_Means = PK_Against_Test.mean()
PK_Against_Test = PK_Against_Test.fillna(ES_Against_Column_Means)
count_nan_in_df = PK_Against_Test.isnull().sum()

PK_Against_Test['xG'] = PK_Against_Coefficients.iat[0,0] + (PK_Against_Test['SA_60']*PK_Against_Coefficients.iat[1,0] + 
                                                            PK_Against_Test['SCSV']*PK_Against_Coefficients.iat[2,0] + 
                                                            PK_Against_Test['HDSA_60']*PK_Against_Coefficients.iat[3,0] +  
                                                            PK_Against_Test['HDGA_60']*PK_Against_Coefficients.iat[4,0] +
                                                            PK_Against_Test['MDSA_60']*PK_Against_Coefficients.iat[5,0] + 
                                                            PK_Against_Test['MDGA_60']*PK_Against_Coefficients.iat[6,0] +
                                                            PK_Against_Test['LDGA_60']*PK_Against_Coefficients.iat[7,0] + 
                                                            PK_Against_Test['SV']*PK_Against_Coefficients.iat[8,0])

print("Expected Goals against with Even Strength:",PK_Against_Test['xG'].mean())
print("True Goals for with Even Strength:",PK_Against_Test['GF_60'].mean())
print("Average difference:", PK_Against_Test['GF_60'].mean()-PK_Against_Test['xG'].mean())


Expected Goals against with Even Strength: 11.399599649768673
True Goals for with Even Strength: 0.9215627411937053
Average difference: -10.478036908574968


In [121]:
################################
# Power Play sample data 
################################
import pyodbc 
pd.set_option('display.max_columns', None)
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-TS4R5T7\SQLEXPRESS;'
                      'Database=NHL;'
                      'Trusted_Connection=yes;')

# cursor = conn.cursor()
# cursor.execute('SELECT * FROM [dbo].[Master Even Strength]')

PP_Against_Sample = pd.read_sql_query('SELECT * FROM [NHL].[dbo].[Master Power Play] WHERE [NHL].[dbo].[Master Power Play].[Date] < '+"'2021-08-08'"+'ORDER BY Year, Month, Day',conn)

PP_Against_Column_Means = PP_Against_Sample.mean()
PP_Against_Sample = PP_Against_Sample.fillna(PP_Against_Column_Means)
count_nan_in_PP_Against_Sample = PP_Against_Sample.isnull().sum()

PP_Against_Sample.head()

,GameID,Location,Opponent,Day,Month,Year,Date,Team,GP,TOI_GP,W,L,OTL,ROW,Points,Point,CF_60,CA_60,CF,FF_60,FA_60,FF,SF_60,SA_60,SF,GF_60,GA_60,GF,xGF_60,xGA_60,xGF,SCF_60,SCA_60,SCF,SCSF_60,SCSA_60,SCSF,SCGF_60,SCGA_60,SCGF,SCSH,SCSV,HDCF_60,HDCA_60,HDCF,HDSF_60,HDSA_60,HDSF,HDGF_60,HDGA_60,HDGF,HDSH,HDSV,MDCF_60,MDCA_60,MDCF,MDSF_60,MDSA_60,MDSF,MDGF_60,MDGA_60,MDGF,MDSH,MDSV,LDCF_60,LDCA_60,LDCF,LDSF_60,LDSA_60,LDSF,LDGF_60,LDGA_60,LDGF,LDSH,LDSV,SH,SV,PDO
0,04-Oct-2017 Toronto Maple Leafs,Away,Winnipeg Jets,4,10,2017,2017-10-04,Toronto Maple Leafs,1,4.8667,True,False,False,True,2,1.0,147.949997,24.660000,85.709999,147.949997,12.33,92.309998,135.619995,12.33,91.669998,24.66,0.0,100.000000,23.02,0.02,99.900002,110.959999,0.0,100.0,110.959999,0.0,100.0,24.66,0.0,100.000000,22.219999,-,73.970001,0.0,100.0,73.970001,0.0,100.0,24.66,0.0,100,33.330002,-,36.990002,0.0,100.000000,36.990002,0.0,100.000000,0.00,0.0,-,0.000000,-,36.990002,12.33,75.000000,24.66,0.0,100.000000,0.0,0.0,-,0.00000,-,18.180000,100.000000,1.182000
1,04-Oct-2017 Winnipeg Jets,Home,Toronto Maple Leafs,4,10,2017,2017-10-04,Winnipeg Jets,1,16.0000,False,True,False,False,0,0.0,116.250000,15.000000,88.570000,82.500000,11.25,88.000000,67.500000,11.25,85.709999,0.00,0.0,87.941603,6.04,0.02,99.730003,63.750000,0.0,100.0,30.000000,0.0,100.0,0.00,0.0,86.754017,0.000000,-,30.000000,0.0,100.0,22.500000,0.0,100.0,0.00,0.0,-,0.000000,-,33.750000,0.0,100.000000,7.500000,0.0,100.000000,0.00,0.0,-,0.000000,-,48.750000,3.75,92.860001,37.50,0.0,100.000000,0.0,0.0,-,0.00000,-,0.000000,100.000000,1.000000
2,04-Oct-2017 St Louis Blues,Away,Pittsburgh Penguins,4,10,2017,2017-10-04,St Louis Blues,1,5.3667,True,False,False,True,2,1.0,55.900002,22.360001,71.430000,55.900002,11.18,83.330002,44.720001,0.00,100.000000,11.18,0.0,100.000000,6.94,0.63,91.650002,44.720001,0.0,100.0,44.720001,0.0,100.0,11.18,0.0,100.000000,25.000000,-,22.360001,0.0,100.0,22.360001,0.0,100.0,0.00,0.0,-,0.000000,-,22.360001,0.0,100.000000,22.360001,0.0,100.000000,11.18,0.0,100,50.000000,-,11.180000,11.18,50.000000,0.00,0.0,91.748383,0.0,0.0,-,8.14863,-,25.000000,90.726481,1.050632
3,04-Oct-2017 Pittsburgh Penguins,Home,St Louis Blues,4,10,2017,2017-10-04,Pittsburgh Penguins,1,6.3167,False,False,True,False,1,0.5,85.489998,9.500000,90.000000,56.990002,9.50,85.709999,28.500000,9.50,75.000000,9.50,0.0,100.000000,9.74,0.62,94.000000,66.489998,9.5,87.5,28.500000,9.5,75.0,9.50,0.0,100.000000,33.330002,100,9.500000,0.0,100.0,9.500000,0.0,100.0,9.50,0.0,100,100.000000,-,56.990002,9.5,85.709999,19.000000,9.5,66.669998,0.00,0.0,-,0.000000,100,19.000000,0.00,100.000000,0.00,0.0,91.748383,0.0,0.0,-,8.14863,-,33.330002,100.000000,1.333000
4,04-Oct-2017 Calgary Flames,Away,Edmonton Oilers,4,10,2017,2017-10-04,Calgary Flames,1,2.0000,False,True,False,False,0,0.0,60.000000,60.000000,50.000000,30.000000,30.00,50.000000,30.000000,0.00,100.000000,0.00,0.0,87.941603,8.71,1.92,81.919998,30.000000,0.0,100.0,30.000000,0.0,100.0,0.00,0.0,86.754017,0.000000,-,30.000000,0.0,100.0,30.000000,0.0,100.0,0.00,0.0,-,0.000000,-,0.000000,0.0,88.299621,0.000000,0.0,86.072079,0.00,0.0,-,14.980242,-,30.000000,30.00,50.000000,0.00,0.0,91.748383,0.0,0.0,-,8.14863,-,0.000000,90.726481,1.050632


In [123]:
###########
# Penalty Kill 
###########

PP_Against_X = PP_Against_Sample[['SA_60','HDSA_60','HDGA_60','MDSA_60','MDGA_60','LDGA_60','SV']]
                        
PP_Against_Y = PP_Against_Sample['GA_60']
 

# with sklearn
PP_Against_Regression = linear_model.LinearRegression()
PP_Against_Regression.fit(PP_Against_X, PP_Against_Y)

#print('Intercept: \n', regr.intercept_)
#print('Coefficients: \n', regr.coef_)

# with statsmodels
PP_Against_X = sm.add_constant(PP_Against_X) # adding a constant
 
PP_Against_Model = sm.OLS(PP_Against_Y, PP_Against_X).fit()
PP_Against_Predictions = PP_Against_Model.predict(PP_Against_X) 
 
PP_Against_Print_Model = PP_Against_Model.summary()
# print(print_model)
# for x in range (0, len(regr.coef_)):
#     print(model.pvalues[x])
PP_Against_Coefficients = pd.DataFrame(PP_Against_Model.params,columns=['COEFFICIENTS'])
print(PP_Against_Print_Model)

                            OLS Regression Results                            
Dep. Variable:                  GA_60   R-squared:                       0.956
Model:                            OLS   Adj. R-squared:                  0.956
Method:                 Least Squares   F-statistic:                 2.759e+04
Date:                Mon, 10 Oct 2022   Prob (F-statistic):               0.00
Time:                        10:27:08   Log-Likelihood:                -10919.
No. Observations:                8822   AIC:                         2.185e+04
Df Residuals:                    8814   BIC:                         2.191e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.0646      0.069     44.541      0.0

In [125]:
###
# Testing analysis against most recent season game data
###
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=DESKTOP-TS4R5T7\SQLEXPRESS;'
                      'Database=NHL;'
                      'Trusted_Connection=yes;')

# cursor = conn.cursor()
# cursor.execute('SELECT * FROM [dbo].[Master Even Strength]')

PP_Against_Test = pd.read_sql_query('SELECT * FROM [NHL].[dbo].[Master Power Play] WHERE [NHL].[dbo].[Master Power Play].[Date] > '+"'2021-08-08'"+'ORDER BY Year, Month, Day',conn)

PP_Against_Column_Means = PP_Against_Test.mean()
PP_Against_Test = PP_Against_Test.fillna(PP_Against_Column_Means)
count_nan_in_df = PP_Against_Test.isnull().sum()

PP_Against_Test['xG'] = PP_Against_Coefficients.iat[0,0] + (PP_Against_Test['SA_60']*PP_Against_Coefficients.iat[1,0] + 
                                                            PP_Against_Test['HDSA_60']*PP_Against_Coefficients.iat[2,0] +  
                                                            PP_Against_Test['HDGA_60']*PP_Against_Coefficients.iat[3,0] +
                                                            PP_Against_Test['MDSA_60']*PP_Against_Coefficients.iat[4,0] + 
                                                            PP_Against_Test['MDGA_60']*PP_Against_Coefficients.iat[5,0] +
                                                            PP_Against_Test['LDGA_60']*PP_Against_Coefficients.iat[6,0] + 
                                                            PP_Against_Test['SV']*PP_Against_Coefficients.iat[7,0])

print("Expected Goals against with Even Strength:",PP_Against_Test['xG'].mean())
print("True Goals for with Even Strength:",PP_Against_Test['GF_60'].mean())
print("Average difference:", PP_Against_Test['GF_60'].mean()-PP_Against_Test['xG'].mean())

Expected Goals against with Even Strength: 0.9414176649783882
True Goals for with Even Strength: 11.15962388155857
Average difference: 10.218206216580182
